In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

from torchvision import transforms, utils
from torchvision.transforms import Resize, ToTensor, Normalize

import time

In [2]:
train_path = 'input/data/train'
train_image_dir_path = os.path.join(train_path, 'images')

Dataset 생성

모든 train data의 path를 가져와 라벨링 진행

In [3]:
def search(dirname, result): # 하위 목록의 모든 파일을 찾는 함수
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            if filename[0] == '.': # .으로 시작하는 애들 거름
                continue
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename, result)
            else:
                ext = os.path.splitext(full_filename)[-1] # 확장자 체크
                if ext:
                    result.append(full_filename)
        
    except PermissionError:
        print('Permission Error')
        pass

In [4]:
all_path = list()
search(train_image_dir_path, all_path)

train의 데이터 디렉토리는 2700개로, 각각의 이미지 파일(incorrect, mask1~5, normal)을 곱한 갯수가 나옵니다.

In [5]:
len(all_path) # 2700 * 7

18900

In [6]:
all_path[:10]

['input/data/train/images/001752_male_Asian_53/mask5.jpg',
 'input/data/train/images/001752_male_Asian_53/mask4.jpg',
 'input/data/train/images/001752_male_Asian_53/mask2.jpg',
 'input/data/train/images/001752_male_Asian_53/mask3.jpg',
 'input/data/train/images/001752_male_Asian_53/normal.jpg',
 'input/data/train/images/001752_male_Asian_53/mask1.jpg',
 'input/data/train/images/001752_male_Asian_53/incorrect_mask.jpg',
 'input/data/train/images/005127_female_Asian_51/mask5.jpg',
 'input/data/train/images/005127_female_Asian_51/mask4.jpg',
 'input/data/train/images/005127_female_Asian_51/mask2.jpg']

파일의 확장자는 jpg, png, jpeg로 3종류가 있습니다.

In [7]:
exts = list()
for word in all_path:
    ext = os.path.splitext(word)[-1]
    if ext not in exts:
        exts.append(ext)
print(exts) # jpg, png, jpeg

['.jpg', '.png', '.jpeg']


In [10]:
all_path = sorted(all_path)

라벨링을 하는 함수입니다. 조건에 따라 label에 숫자를 더해주는 식으로 만들었습니다.

In [11]:
def labeling(name):
    label = 0
    info, mask_type = name.split('/')[-2:]
    info = info.split('_')
    gender, age = info[1], int(info[3])
    
    # 마스크 구별
    if 'incorrect' in mask_type:
        label += 6
    elif 'normal' in mask_type:
        label += 12
    
    # gender 구별
    if gender == 'female':
        label += 3
    
    # 나이 구별
    if 30 <= age and age < 60:
        label += 1
    elif age >= 60:
        label += 1
    
    return label

path, label을 컬럼으로 갖는 dataframe을 생성해줍니다.

In [12]:
train_path_label = pd.DataFrame(all_path, columns = ['path'])

train_path_label['label'] = train_path_label['path'].map(lambda x: labeling(x))
train_path_label

,path,label
0,input/data/train/images/000001_female_Asian_45...,10
1,input/data/train/images/000001_female_Asian_45...,4
2,input/data/train/images/000001_female_Asian_45...,4
3,input/data/train/images/000001_female_Asian_45...,4
4,input/data/train/images/000001_female_Asian_45...,4
...,...,...
18895,input/data/train/images/006959_male_Asian_19/m...,0
18896,input/data/train/images/006959_male_Asian_19/m...,0
18897,input/data/train/images/006959_male_Asian_19/m...,0
18898,input/data/train/images/006959_male_Asian_19/m...,0


In [ ]:
# train_path_label.to_csv('./train_path_label.csv', index=False, encoding='utf-8')
# train_path_label = pd.read_csv('./train_path_label.csv', encoding='utf-8')

dataset을 상속받아 만든 CustomDataset입니다. transform은 size를 [512, 384]로 변형하고, Tensor로 만들고, 정규화를 해주었습니다.

In [13]:
class CustomDataset(Dataset):
    def __init__(self, img_paths_label, transform):
        self.X = img_paths_label['path']
        self.y = img_paths_label['label']
        self.transform = transform
        
    def __getitem__(self, index):
        image = Image.open(self.X.iloc[index])
        label = self.y.iloc[index]
        
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)
    
    def __len__(self):
        return len(self.X)

In [14]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

train, valid를 나누는 부분입니다.

label의 비율을 유지하면서 나눴습니다.

+ 기존 방법 대신StratifiedKFold 사용을 시도했습니다.

In [ ]:
# 원본
# from sklearn.model_selection import train_test_split
# train, valid = train_test_split(train_path_label, test_size=0.2,
#                                shuffle=True, stratify=train_path_label['label'],
#                                random_state=34)

In [52]:
# 5-fold Stratified KFold 5개의 fold를 형성하고 5번 Cross Validation을 진행합니다.
from sklearn.model_selection import StratifiedKFold

# skf 설정
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)
# skf에서 사용할 labels 설정
labels = [i for i in train_path_label['label']]

train = list()
valid = list()
# 각 클래스 균등하게 나누는 StratifiedKFold로 class 등분해서 cross-validation
for i, (train_idx, valid_idx) in enumerate(skf.split(train_path_label, labels)):
    print(type(train_idx))
    train.append(train_idx)
    valid.append(valid_idx)
#     print(train_idx, valid_idx)
#     print(len(train_idx), len(valid_idx))
    
print(train[:10])

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[array([ 1627,  1628,  1629, ..., 18897, 18898, 18899]), array([    0,     1,     2, ..., 18897, 18898, 18899]), array([    0,     1,     2, ..., 18897, 18898, 18899]), array([    0,     1,     2, ..., 18897, 18898, 18899]), array([    0,     1,     2, ..., 18044, 18045, 18047])]


In [50]:
train[0].shape()

TypeError: 'tuple' object is not callable

dataloader를 정의했습니다. batchsize는 64로 했고 shuffle을 했습니다.

In [ ]:
BATCH_SIZE = 64

In [ ]:
custom_dataset = CustomDataset(train, transform)

custom_dataloader = DataLoader(train_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True
                             )